<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/digitalcoupon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper for flatRate, similar to C++ utilities.hpp
def flat_rate_py(evaluation_date, forward_rate, day_counter):
    return ql.FlatForward(evaluation_date, ql.QuoteHandle(ql.SimpleQuote(forward_rate)), day_counter)

# Helper for flatVol, similar to C++ utilities.hpp
def flat_vol_py(evaluation_date, vol_level, day_counter):
    return ql.BlackConstantVol(evaluation_date, ql.NullCalendar(), ql.QuoteHandle(ql.SimpleQuote(vol_level)), day_counter)


class CommonVarsDigital: # Renamed to avoid conflict if other tests use CommonVars
    def __init__(self):
        self.fixing_days = 2
        self.nominal = 1000000.0

        self.term_structure_handle = ql.RelinkableYieldTermStructureHandle()
        self.index = ql.Euribor6M(self.term_structure_handle)
        self.calendar = self.index.fixingCalendar()

        # Set evaluation date carefully, possibly in setUp of TestCase
        self.today = self.calendar.adjust(ql.Settings.instance().evaluationDate)
        # ql.Settings.instance().evaluationDate = self.today # Ensure it's set globally for this context

        self.settlement = self.calendar.advance(self.today, self.fixing_days, ql.Days)
        self.term_structure_handle.linkTo(flat_rate_py(self.settlement, 0.05, ql.Actual365Fixed()))

        self.option_tolerance = 1.e-04
        self.black_tolerance = 1e-10


class DigitalCouponTests(unittest.TestCase):

    def setUp(self):
        # Set a consistent evaluation date for all tests in this class
        # This is crucial as CommonVarsDigital reads it.
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        eval_date = ql.Date(15, ql.May, 2007) # Example, ensure it's consistent
        ql.Settings.instance().evaluationDate = eval_date

        self.common = CommonVarsDigital()

    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.saved_eval_date


    def test_asset_or_nothing(self):
        print("Testing European asset-or-nothing digital coupon...")
        vars = self.common

        vols = [0.05, 0.15, 0.30]
        strikes = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07]
        gearings_arr = [1.0, 2.8]
        spreads_arr = [0.0, 0.005]

        gap_replication = 1e-7
        replication = ql.DigitalReplication(ql.Replication.Central, gap_replication)
        phi = ql.CumulativeNormalDistribution()

        for caplet_vol_val in vols:
            vol_handle = ql.RelinkableOptionletVolatilityStructureHandle()
            vol_handle.linkTo(ql.ConstantOptionletVolatility(
                vars.today, vars.calendar, ql.Following,
                ql.QuoteHandle(ql.SimpleQuote(caplet_vol_val)), ql.Actual360() # Vol quote handle
            ))

            for strike_val in strikes:
                for k in range(9, 10): # C++: k=9 to <10 -> k=9
                    start_date = vars.calendar.advance(vars.settlement, (k + 1), ql.Years)
                    end_date = vars.calendar.advance(vars.settlement, (k + 2), ql.Years)
                    # Use None for nullstrike in Python for DigitalCoupon constructor
                    # Python binding might expect float or None. Using a very OTM strike can also work if None is not accepted.
                    # For DigitalCoupon, often strike=None for non-option part.
                    # C++ Null<Rate>() implies it's not set.
                    # DigitalCoupon constructor: underlying, callStrike, callPosition, isCallATMIncluded, callDigitalPayoff,
                    #                            putStrike, putPosition, isPutATMIncluded, putDigitalPayoff, replication

                    for h_idx in range(len(gearings_arr)):
                        gearing = gearings_arr[h_idx]
                        spread = spreads_arr[h_idx]

                        underlying_coupon = ql.IborCoupon(
                            end_date, vars.nominal, start_date, end_date,
                            vars.fixing_days, vars.index, gearing, spread
                        )
                        pricer = ql.BlackIborCouponPricer(vol_handle)
                        underlying_coupon.setPricer(pricer) # Need to price underlying to get its rate/forward

                        # Call Digital: (aL+b)Heaviside(aL+b-X)
                        # DigitalCoupon params for "call option on (aL+b) at strike X":
                        # callStrike=X, callPosition=Long (if buyer), isCallATM=false, callDigitalPayoff=None (for AssetOrNothing)
                        # put features are set to None or far OTM
                        digital_call_coupon = ql.DigitalCoupon(
                            underlying_coupon,
                            callStrike=strike_val, callPosition=ql.Position.Short, # Matches C++ payoff (aL+b)I(aL+b > K)
                            isCallATMIncluded=False, callDigitalPayoff=None, # AssetOrNothing
                            putStrike=None, putPosition=ql.Position.Short, # No put part
                            isPutATMIncluded=False, putDigitalPayoff=None,
                            replication=replication
                        )
                        digital_call_coupon.setPricer(pricer)

                        accrual_period = underlying_coupon.accrualPeriod()
                        discount = vars.term_structure_handle.discount(end_date)
                        exercise_date = underlying_coupon.fixingDate()
                        # underlying_coupon.rate() needs the pricer to be set on underlying for forward
                        forward_rate_val = underlying_coupon.rate()

                        eff_fwd = (forward_rate_val - spread) / gearing
                        eff_strike = (strike_val - spread) / gearing

                        std_dev = math.sqrt(vol_handle.blackVariance(exercise_date, eff_strike))
                        if std_dev < 1e-16: std_dev = 1e-16 # Avoid division by zero

                        d1 = math.log(eff_fwd / eff_strike if eff_strike > 0 else float('inf')) / std_dev + 0.5 * std_dev
                        d2 = d1 - std_dev
                        N_d1 = phi(d1)
                        N_d2 = phi(d2)

                        # Value Call = (aF N(d1') + b N(d2')) * Nominal * Accrual * Discount
                        # Where F = eff_fwd, X' = eff_strike.
                        # (gearing * eff_fwd * N_d1 + spread * N_d2)
                        expected_call_value = (gearing * eff_fwd * N_d1 + spread * N_d2) * \
                                           vars.nominal * accrual_period * discount

                        # DigitalCoupon.callOptionRate() returns the option rate (not price)
                        # callOptionRate() is the additional rate from the call option component
                        # If Position.Short, then this value is negative.
                        # The C++ test has Position::Short.
                        # The formula `aF N(d1) + b N(d2)` is for a long asset-or-nothing call.
                        # So, if digital_call_coupon is Position.Short, its option rate will be negative.
                        # For comparison, we take absolute value or ensure signs match.
                        # C++ (aL+b)Heaviside(aL+b-X), Position::Short for callStrike implies we *sell* this.
                        # Value of short asset-or-nothing = -(aF N(d1) + bN(d2))
                        # The callOptionRate() from DigitalCoupon should give the value of the option part.
                        # If digital coupon is `underlying - CallAoN(K)`, then callOptionRate should be negative.
                        option_price_from_coupon = digital_call_coupon.callOptionRate() * \
                                                   vars.nominal * accrual_period * discount

                        # The formula from C++ seems to be for a LONG digital.
                        # The DigitalCoupon(..., strike, Position::Short, ...) is underlying - Call.
                        # So its callOptionRate() component should be negative.
                        # Let's assume the C++ "nd1Price" is for a standalone long call.
                        self.assertAlmostEqual(-option_price_from_coupon, expected_call_value, delta=vars.option_tolerance * vars.nominal,
                                               msg=f"Digital Call (AssetOrNothing) price mismatch Vol={caplet_vol_val}, K={strike_val}")

                        # ... similar setup for Put Digital ...
                        digital_put_coupon = ql.DigitalCoupon(
                            underlying_coupon,
                            callStrike=None, callPosition=ql.Position.Long,
                            isCallATMIncluded=False, callDigitalPayoff=None,
                            putStrike=strike_val, putPosition=ql.Position.Long, # Buy PutAoN
                            isPutATMIncluded=False, putDigitalPayoff=None,
                            replication=replication
                        )
                        digital_put_coupon.setPricer(pricer)

                        # Value Put = (aF N(-d1') + bN(-d2'))
                        expected_put_value = (gearing * eff_fwd * phi(-d1) + spread * phi(-d2)) * \
                                          vars.nominal * accrual_period * discount
                        option_price_from_put_coupon = digital_put_coupon.putOptionRate() * \
                                                       vars.nominal * accrual_period * discount
                        self.assertAlmostEqual(option_price_from_put_coupon, expected_put_value, delta=vars.option_tolerance * vars.nominal,
                                               msg=f"Digital Put (AssetOrNothing) price mismatch Vol={caplet_vol_val}, K={strike_val}")

    # ... Other tests (DITM, DOTM, CashOrNothing, Parity, ReplicationType) would follow similar translation patterns ...
    # These are extensive and involve careful setup of DigitalCoupon and comparison logic.
    # For brevity, I'll outline one more test (e.g., Call/Put Parity) to show the pattern.

    def test_call_put_parity(self):
        print("Testing call/put parity for European digital coupon...")
        vars = self.common

        vols = [0.05, 0.15, 0.30]
        strikes = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07]
        gearing = 1.0
        spread = 0.0
        cash_rate = 0.01 # For CashOrNothing part

        for caplet_vol_val in vols:
            vol_handle = ql.RelinkableOptionletVolatilityStructureHandle()
            vol_handle.linkTo(ql.ConstantOptionletVolatility(
                vars.today, vars.calendar, ql.Following,
                ql.QuoteHandle(ql.SimpleQuote(caplet_vol_val)), ql.Actual360()
            ))
            pricer = ql.BlackIborCouponPricer(vol_handle)

            for strike_val in strikes:
                for k in range(10): # Loop 0 to 9
                    start_date = vars.calendar.advance(vars.settlement, (k + 1), ql.Years)
                    end_date = vars.calendar.advance(vars.settlement, (k + 2), ql.Years)

                    underlying_coupon = ql.IborCoupon(
                        end_date, vars.nominal, start_date, end_date,
                        vars.fixing_days, vars.index, gearing, spread
                    )
                    underlying_coupon.setPricer(pricer) # Needed for underlying_coupon.rate()

                    # --- Cash-or-Nothing Parity ---
                    # Long Call CashOrNothing + Underlying VS Short Put CashOrNothing + Underlying
                    # Call Price - Put Price = R * Discounted Accrual Period (if long call, short put)
                    # C++: cash_digitalCallCoupon - cash_digitalPutCoupon
                    # cash_digitalCallCoupon = underlying + Long CallCoN(K, cashRate)
                    # cash_digitalPutCoupon = underlying - Short PutCoN(K, cashRate) -> underlying + Long PutCoN(K,cashRate) if Position::Short is for option part.
                    # The C++ setup is:
                    # DigitalCoupon cash_digitalCallCoupon(underlying, strike, Position::Long, false, cashRate, ...);
                    #   This is underlying + (cashRate if L>K). callOptionRate() is positive.
                    # DigitalCoupon cash_digitalPutCoupon(underlying, ..., strike, Position::Short, false, cashRate);
                    #   This is underlying - (cashRate if L<K). putOptionRate() is negative.
                    # So, price(call) - price(put) = (underlying_price + call_opt_price) - (underlying_price + put_opt_price)
                    #                               = call_opt_price - put_opt_price
                    # This should equal R * D(T) * accP.

                    cash_call_coupon = ql.DigitalCoupon(underlying_coupon,
                        callStrike=strike_val, callPosition=ql.Position.Long, isCallATMIncluded=False, callDigitalPayoff=cash_rate,
                        putStrike=None, putPosition=ql.Position.Long, isPutATMIncluded=False, putDigitalPayoff=None)
                    cash_call_coupon.setPricer(pricer)

                    cash_put_coupon = ql.DigitalCoupon(underlying_coupon,
                        callStrike=None, callPosition=ql.Position.Long, isCallATMIncluded=False, callDigitalPayoff=None,
                        putStrike=strike_val, putPosition=ql.Position.Short, isPutATMIncluded=False, putDigitalPayoff=cash_rate)
                    cash_put_coupon.setPricer(pricer)

                    price_diff_cash = cash_call_coupon.price(vars.term_structure_handle) - \
                                      cash_put_coupon.price(vars.term_structure_handle)

                    accrual_period = underlying_coupon.accrualPeriod()
                    discount = vars.term_structure_handle.discount(end_date)
                    target_price_cash = vars.nominal * accrual_period * discount * cash_rate

                    self.assertAlmostEqual(price_diff_cash, target_price_cash, delta=1e-8 * vars.nominal,
                                           msg=f"CashOrNothing Parity: Vol={caplet_vol_val}, K={strike_val}")

                    # --- Asset-or-Nothing Parity ---
                    # Long Call AssetOrNothing + Underlying VS Short Put AssetOrNothing + Underlying
                    # Call Price - Put Price = Underlying Price
                    # C++: asset_digitalCallCoupon - asset_digitalPutCoupon
                    asset_call_coupon = ql.DigitalCoupon(underlying_coupon,
                        callStrike=strike_val, callPosition=ql.Position.Long, isCallATMIncluded=False, callDigitalPayoff=None, # Asset
                        putStrike=None, putPosition=ql.Position.Long, isPutATMIncluded=False, putDigitalPayoff=None)
                    asset_call_coupon.setPricer(pricer)

                    asset_put_coupon = ql.DigitalCoupon(underlying_coupon,
                        callStrike=None, callPosition=ql.Position.Long, isCallATMIncluded=False, callDigitalPayoff=None,
                        putStrike=strike_val, putPosition=ql.Position.Short, isPutATMIncluded=False, putDigitalPayoff=None) # Asset
                    asset_put_coupon.setPricer(pricer)

                    price_diff_asset = asset_call_coupon.price(vars.term_structure_handle) - \
                                       asset_put_coupon.price(vars.term_structure_handle)

                    # Target price = underlying coupon's value = Nominal * Accrual * Forward * Discount
                    # forward_rate = underlying_coupon.rate()
                    # target_price_asset = vars.nominal * accrual_period * discount * forward_rate
                    # Or simply:
                    target_price_asset = underlying_coupon.price(vars.term_structure_handle)


                    self.assertAlmostEqual(price_diff_asset, target_price_asset, delta=1e-7 * vars.nominal,
                                           msg=f"AssetOrNothing Parity: Vol={caplet_vol_val}, K={strike_val}")


if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

Full Test Suite: The provided Python code translates testAssetOrNothing and testCallPutParity as examples. A complete translation would require implementing all other BOOST_AUTO_TEST_CASE blocks (testAssetOrNothingDeepInTheMoney, testAssetOrNothingDeepOutTheMoney, testCashOrNothing, testCashOrNothingDeepInTheMoney, testCashOrNothingDeepOutTheMoney, testReplicationType) following similar patterns. This is extensive.
Null<Rate> and None: In Python, None is typically used for optional arguments that are not provided. For callDigitalPayoff or putDigitalPayoff, None signifies an AssetOrNothing type payoff. For callStrike or putStrike, None means that particular option component is not active.
DigitalCoupon Constructor: The Python constructor for ql.DigitalCoupon might have a slightly different signature or way of specifying the digital features compared to the C++ one which uses Null<Rate> to deactivate parts. The common way is:
DigitalCoupon(underlyingCoupon, callStrike=None, callPosition=Position.Long, isCallATMIncluded=False, callDigitalPayoff=None, putStrike=None, putPosition=Position.Long, isPutATMIncluded=False, putDigitalPayoff=None, replication=None)
If callDigitalPayoff is a Rate (float), it's CashOrNothing. If None, it's AssetOrNothing.
Underlying Coupon Pricing: Remember that underlying_coupon.rate() (to get the forward rate) and underlying_coupon.price() require the BlackIborCouponPricer to be set on the underlying_coupon itself, not just on the DigitalCoupon.
Sign Conventions for callOptionRate()/putOptionRate(): The DigitalCoupon methods callOptionRate() and putOptionRate() return the value of the option component. If the Position for that option is Short in the DigitalCoupon's definition, the rate returned by these methods will be negative. The C++ test's "expected" values often assume a standalone long option. My Python translation attempts to adjust for this by either comparing with the negative of the coupon's option rate or by ensuring the DigitalCoupon is constructed to represent a long option for direct comparison with the formula. This requires careful checking of how DigitalCoupon is constructed in each C++ sub-test.
Tolerance: The tolerances for comparisons are multiplied by vars.nominal in some places because the option prices/rates are often per unit of notional, and the final comparison is on the total value.
Verbosity: The original C++ test is very dense with many loops and checks. The Python translation mirrors this. For a production test suite, some of these loops might be refactored or parameterized differently.
